In [ ]:
### Preliminars
#### Libraries
install.packages('dplyr')
install.packages('ggplot2')
install.packages('corrplot')
install.packages('GGally')
install.packages('car')
install.packages('MASS')
install.packages('stringr')
install.packages('gridExtra')
install.packages('rpart')
install.packages('rattle')

library(dplyr)
library(ggplot2)
library(corrplot)
library(GGally)
library(car)
library(MASS)
library(stringr)
library(gridExtra)
library(rpart)
library(rattle)

#### Upload the database
df <- read.csv("base_preproc_4.csv", header=TRUE, sep = ';', encoding = 'latin1')

### Data pre-processing

colnames(df) <- c("año_de_presentacion","expediente","caratula","fecha_de_resolucion","cita_de_fallos",
  "tipo_de_actor_1", "tipo_de_actor_2", "tipo_de_actor_3", "vecinos_de_otra_provincia", "corte_causal_vecindad",
  "tipo_de_demandado_1", "tipo_de_demandado_2", "tipo_de_demandado_3", "tipo_de_demandado_4", "cantidad_de_provincias",
  "citacion_de_terceros", "provincias_citadas_terceros_tribunal", "tema", "recurso", "posterga_decision","decision_de_la_corte", "argumento",
  "involucra_recurso_interjur", "prueba_daño_interjur", "prueba_presentada", "dicta_cautelar", "audiencia_publica",
  "conformacion_mayoria", "disidencia_voto", "disidencia_sobre_total","cantidad_jueces_votan", "jueces_no_votaron", "presidente", "composicion",
  "jurisprudencia_citada", "decision_procurador", "involucra_recurso_interjur_procurador", "remision_al_procurador",
  "secretaria", "comentarios")

#### Coding variables
df$fecha_de_resolucion <- as.integer(str_sub(df$fecha_de_resolucion,-4,-1))
df$tiempo_de_resolucion <- df$fecha_de_resolucion - df$año_de_presentacion

df$cita_de_fallos_dummy = ifelse(df$cita_de_fallos=='No tiene',0,1)
df$cita_de_fallos = ifelse(df$cita_de_fallos == 'No tiene', NA, df$cita_de_fallos)

# number of actors
df$cantidad_de_actores_1 <- ifelse(df$tipo_de_actor_1 == "NA",0,1)
df$cantidad_de_actores_2 <- ifelse(df$tipo_de_actor_2=="NA",0,1)
df$cantidad_de_actores_3 <- ifelse(df$tipo_de_actor_3=="NA",0,1)
df$cantidad_de_actores <- rowSums(cbind(df$cantidad_de_actores_1, df$cantidad_de_actores_2, df$cantidad_de_actores_3),na.rm=TRUE)
df <- df[,!names(df) %in% c("cantidad_de_actores_1", "cantidad_de_actores_2", "cantidad_de_actores_3")]

# Number of plaintiffs
df$cantidad_de_demandados_1 <- ifelse(df$tipo_de_demandado_1== "NA",0,1)
df$cantidad_de_demandados_2 <- ifelse(df$tipo_de_demandado_2=="NA",0,1)
df$cantidad_de_demandados_3 <- ifelse(df$tipo_de_demandado_3=="NA",0,1)
df$cantidad_de_demandados_4 <- ifelse(df$tipo_de_demandado_4=="NA",0,1)
df$cantidad_de_demandados <- rowSums(cbind(df$cantidad_de_demandados_1,
  df$cantidad_de_demandados_2,
  df$cantidad_de_demandados_3,
  df$cantidad_de_demandados_4),
  na.rm=TRUE)
df <- df[,!names(df) %in% c("cantidad_de_demandados_1", "cantidad_de_demandados_2", "cantidad_de_demandados_3","cantidad_de_demandados_4")]

# number of provinces
df = df %>% mutate(cantidad_de_provincias = case_when(cantidad_de_provincias > 0 ~ cantidad_de_provincias, TRUE ~ 0))

# features concerning the actors
df$actor_empresa <- ifelse(tolower(df$tipo_de_actor_1) == "empresa" |
  tolower(df$tipo_de_actor_2) == "empresa" | tolower(df$tipo_de_actor_3) == "empresa",1,0)
df$actor_empresa[is.na(df$actor_empresa)] <- 0

df$actor_ente_nac <- ifelse(tolower(df$tipo_de_actor_1) == "ente nacional/internacional" |
  tolower(df$tipo_de_actor_2) == "ente nacional/internacional" | tolower(df$tipo_de_actor_3) == "ente nacional/internacional",1,0)
df$actor_ente_nac[is.na(df$actor_ente_nac)] <- 0

df$actor_ente_prov <- ifelse(tolower(df$tipo_de_actor_1) == "ente provincial" |
  tolower(df$tipo_de_actor_2) == "ente provincial" | tolower(df$tipo_de_actor_3) == "ente provincial",1,0)
df$actor_ente_prov[is.na(df$actor_ente_prov)] <- 0

df$actor_estado_nac <- ifelse(tolower(df$tipo_de_actor_1) == "estado nacional" |
  tolower(df$tipo_de_actor_2) == "estado nacional" | tolower(df$tipo_de_actor_3) == "estado nacional",1,0)
df$actor_estado_nac[is.na(df$actor_estado_nac)] <- 0

df$actor_estado_prov <- ifelse(tolower(df$tipo_de_actor_1) == "estado provincial" |
  tolower(df$tipo_de_actor_2) == "estado provincial" | tolower(df$tipo_de_actor_3) == "estado provincial",1,0)
df$actor_estado_prov[is.na(df$actor_estado_prov)] <- 0

df$actor_mpfn <- ifelse(tolower(df$tipo_de_actor_1) == "ministerio público fiscal de la nación" |
  tolower(df$tipo_de_actor_2) == "ministerio público fiscal de la nación" | tolower(df$tipo_de_actor_3) == "ministerio público fiscal de la nación",1,0)
df$actor_mpfn[is.na(df$actor_mpfn)] <- 0

df$actor_municipio <- ifelse(tolower(df$tipo_de_actor_1) == "municipio" |
  tolower(df$tipo_de_actor_2) == "municipio" | tolower(df$tipo_de_actor_3) == "municipio",1,0)
df$actor_municipio[is.na(df$actor_municipio)] <- 0

df$actor_ong <- ifelse(tolower(df$tipo_de_actor_1) == "ong" |
  tolower(df$tipo_de_actor_2) == "ong" | tolower(df$tipo_de_actor_3) == "ong",1,0)
df$actor_ong[is.na(df$actor_ong)] <- 0

df$actor_pers_fisica <- ifelse(tolower(df$tipo_de_actor_1) == "persona física" |
  tolower(df$tipo_de_actor_2) == "persona física" | tolower(df$tipo_de_actor_3) == "persona física",1,0)
df$actor_pers_fisica[is.na(df$actor_pers_fisica)] <- 0

df$actor_pue_orig <- ifelse(tolower(df$tipo_de_actor_1) == "pueblos originarios" |
  tolower(df$tipo_de_actor_2) == "pueblos originarios" | tolower(df$tipo_de_actor_3) == "pueblos originarios",1,0)
df$actor_pue_orig[is.na(df$actor_pue_orig)] <- 0

# features concerning the plaintiffs
df$demandado_caba <- ifelse(df$tipo_de_demandado_1 == "Ciudad Autónoma de Buenos Aires" |
  df$tipo_de_demandado_2 == "Ciudad Autónoma de Buenos Aires" | df$tipo_de_demandado_3 == "Ciudad Autónoma de Buenos Aires" |
  df$tipo_de_demandado_4 == 'Ciudad Autónoma de Buenos Aires',1,0)
df$demandado_caba[is.na(df$demandado_caba)] <- 0

df$demandado_empresa <- ifelse(df$tipo_de_demandado_1 == "Empresa" |
  df$tipo_de_demandado_2 == "Empresa" | df$tipo_de_demandado_3 == "Empresa" |
  df$tipo_de_demandado_4 == "Empresa",1,0)
df$demandado_empresa[is.na(df$demandado_empresa)] <- 0

df$demandado_ente_nac <- ifelse(df$tipo_de_demandado_1 == "Ente nacional/internacional" |
  df$tipo_de_demandado_2 == "Ente nacional/internacional" | df$tipo_de_demandado_3 == "Ente nacional/internacional" |
  df$tipo_de_demandado_4 == "Ente nacional/internacional",1,0)
df$demandado_ente_nac[is.na(df$demandado_ente_nac)] <- 0

df$demandado_ente_prov <- ifelse(df$tipo_de_demandado_1 == "Ente provincial" |
  df$tipo_de_demandado_2 == "Ente provincial" | df$tipo_de_demandado_3 == "Ente provincial" |
  df$tipo_de_demandado_4 == "Ente provincial",1,0)
df$demandado_ente_prov[is.na(df$demandado_ente_prov)] <- 0

df$demandado_estado_nac <- ifelse(df$tipo_de_demandado_1 == "Estado Nacional" |
  df$tipo_de_demandado_2 == "Estado Nacional" | df$tipo_de_demandado_3 == "Estado Nacional" |
  df$tipo_de_demandado_4 == "Estado Nacional",1,0)
df$demandado_estado_nac[is.na(df$demandado_estado_nac)] <- 0

df$demandado_estado_prov <- ifelse(df$tipo_de_demandado_1 == "Estado Provincial" |
  df$tipo_de_demandado_2 == "Estado Provincial" | df$tipo_de_demandado_3 == "Estado Provincial" |
  df$tipo_de_demandado_4 == "Estado Provincial",1,0)
df$demandado_estado_prov[is.na(df$demandado_estado_prov)] <- 0

df$demandado_municipio <- ifelse(df$tipo_de_demandado_1 == "Municipio" |
  df$tipo_de_demandado_2 == "Municipio" | df$tipo_de_demandado_3 == "Municipio" |
  df$tipo_de_demandado_4 == "Municipio",1,0)
df$demandado_municipio[is.na(df$demandado_municipio)] <- 0

df$demandado_actor_interjuris <- ifelse(df$tipo_de_demandado_1 == "Organismo Interjurisdiccional" |
  df$tipo_de_demandado_2 == "Organismo Interjurisdiccional" | df$tipo_de_demandado_3 == "Organismo Interjurisdiccional" |
  df$tipo_de_demandado_4 == "Organismo Interjurisdiccional",1,0)
df$demandado_actor_interjuris[is.na(df$demandado_actor_interjuris)] <- 0

df$demandado_pers_fisica <- ifelse(df$tipo_de_demandado_1 == "Persona Física" |
  df$tipo_de_demandado_2 == "Persona Física" | df$tipo_de_demandado_3 == "Persona Física" |
  df$tipo_de_demandado_4 == "Persona Física",1,0)
df$demandado_pers_fisica[is.na(df$demandado_pers_fisica)] <- 0

df$demandado_empresa_estatal <- ifelse(df$tipo_de_demandado_1 == "Empresa Estatal" |
  df$tipo_de_demandado_2 == "Empresa Estatal" | df$tipo_de_demandado_3 == "Empresa Estatal" |
  df$tipo_de_demandado_4 == "Empresa Estatal",1,0)
df$demandado_empresa_estatal[is.na(df$demandado_empresa_estatal)] <- 0

# vecinos de otra provincia
df = df %>% mutate(vecinos_de_otra_provincia_1 = case_when(tolower(vecinos_de_otra_provincia) == "si" ~ 0,
                           vecinos_de_otra_provincia == "No" ~ 1,
                           vecinos_de_otra_provincia == "No dice" ~ 1,
                           TRUE ~ NA))
df = df %>% mutate(vecinos_de_otra_provincia_2 = case_when(tolower(vecinos_de_otra_provincia) == "si" ~ 0,
                           vecinos_de_otra_provincia == "No" ~ 1,
                           vecinos_de_otra_provincia == "No dice" ~ 0,
                           TRUE ~ NA))

# citación de terceros
df = df %>% mutate(citacion_de_terceros_1 = case_when(tolower(citacion_de_terceros) == "si" ~ 1,
                          tolower(citacion_de_terceros) == "no" ~ 0,
                          tolower(citacion_de_terceros) == "rechaza" ~ 1))
df = df %>% mutate(citacion_de_terceros_2 = case_when(tolower(citacion_de_terceros) == "si" ~ 1,
                          tolower(citacion_de_terceros) == "rechaza" ~ 0,
                          TRUE ~ NA))

# provinces cited as third parties by the court
df = df %>% mutate(provincias_citadas_terceros_tribunal_2 = case_when(tolower(citacion_de_terceros) == "si" ~ provincias_citadas_terceros_tribunal,
                        tolower(citacion_de_terceros) == "no" ~ NA,
                        tolower(citacion_de_terceros) == "rechaza" ~ NA))

# subject
df$tema <- factor(df$tema)

# dummy variable indicating if involves interjurisdictional remedy according to the court
df$involucra_recurso_interjur <- ifelse(df$involucra_recurso_interjur == "Remite PGN: No","No", df$involucra_recurso_interjur)
df = df %>% mutate(involucra_recurso_interjur = case_when(involucra_recurso_interjur == "Si" ~ 1,
                           involucra_recurso_interjur == "No" ~ 0,
                           involucra_recurso_interjur == "No dice" ~ NA,
                           TRUE ~ NA))

# dummy variable indicating proof of interjurisdictional damage
df = df %>% mutate(prueba_daño_interjur_2 = as.factor(case_when(prueba_daño_interjur == "Argumenta que por la posición de la planta que vuelca los deshechos cloacales, el problemas es claramente interjurisdiccional" ~ "Si",
                           prueba_daño_interjur == "Remite PGN" ~ "No",
                           prueba_daño_interjur == "No" ~ "No",
                           prueba_daño_interjur == "Si" ~ "Si",
                           prueba_daño_interjur == "Insuficiente" ~ "Insuficiente",
                           prueba_daño_interjur == "No dice" ~ "No dice",
                           TRUE ~ "NA")))

# dummy variable indicating whether it issues an interim injunction
df = df %>% mutate(dicta_cautelar = case_when(dicta_cautelar == "Si" ~ 1,
                           dicta_cautelar == "No" ~ 0,
                           dicta_cautelar == "Difiere" ~ NA))

# dummy variable indicating public hearing
df$audiencia_publica <- ifelse(tolower(df$audiencia_publica)=='si',1,0)

# majority voting
df$conformacion_mayoria = as.factor(df$conformacion_mayoria)

# whose dissent / vote:
df = df %>% mutate(disidencia_argibay = case_when((conformacion_mayoria == 'Disidencia' & grepl("Argibay", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(disidencia_highton = case_when((conformacion_mayoria == 'Disidencia' & grepl("Highton", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(disidencia_lorenzetti = case_when((conformacion_mayoria == 'Disidencia' & grepl("Lorenzetti", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_fayt = case_when((conformacion_mayoria == 'Voto' & grepl("Fayt", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_petracchi = case_when((conformacion_mayoria == 'Voto' & grepl("Pettrachi", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_highton = case_when((conformacion_mayoria == 'Voto' & grepl("Highton", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_argibay = case_when((conformacion_mayoria == 'Voto' & grepl("Argibay", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_maqueda = case_when((conformacion_mayoria == 'Voto' & grepl("Maqueda", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_zaffaroni = case_when((conformacion_mayoria == 'Voto' & grepl("Zaffaroni", disidencia_voto)) ~ 1, TRUE ~ 0))
df = df %>% mutate(voto_rosenkrantz = case_when((conformacion_mayoria == 'Voto' & grepl("Rosenkrantz", disidencia_voto)) ~ 1, TRUE ~ 0))

# who does not vote:
df = df %>% mutate(no_vota_argibay = case_when(grepl("argibay", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_boggiano = case_when(grepl("boggiano", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_bossert = case_when(grepl("bossert", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_fayt = case_when(grepl("fayt", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_highton = case_when(grepl("highton", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_levene = case_when(grepl("levene", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_lopez = case_when(grepl("lopez", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_lorenzetti = case_when(grepl("lorenzetti", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_maqueda = case_when(grepl("maqueda", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_moline = case_when(grepl("moliné", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_nazareno = case_when(grepl("nazareno", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_petracchi = case_when(grepl("petracchi", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_rosatti = case_when(grepl("rosatti", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_rosenkrantz = case_when(grepl("rosenkrantz", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_vazquez = case_when(grepl("vazquez", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))
df = df %>% mutate(no_vota_zaffaroni = case_when(grepl("zaffaroni", tolower(jueces_no_votaron)) ~ 1, TRUE ~ 0))

# president of the court
df$presidente <- factor(df$presidente)

# court composition
df = df %>% mutate(composicion_cr1 = case_when(presidente == "CR" ~ 1, TRUE ~ NA))
df = df %>% mutate(composicion_emo1 = case_when(presidente == "EMO" ~ 1, TRUE ~ NA))
df = df %>% mutate(composicion_ep1 = case_when(presidente == "EP" & composicion == "EP2" ~ 1,
  presidente == "EP" & composicion != "EP2" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_ep2 = case_when(presidente == "EP" & composicion == "EP3" ~ 1,
  presidente == "EP" & composicion != "EP3" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_ep3 = case_when(presidente == "EP" & composicion == "EP6" ~ 1,
  presidente == "EP" & composicion != "EP6" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_ep4 = case_when(presidente == "EP" & composicion == "EP7" ~ 1,
  presidente == "EP" & composicion != "EP7" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_hr1 = case_when(presidente == "HR" & composicion == "HR1" ~ 1,
  presidente == "HR" & composicion != "HR1" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_hr2 = case_when(presidente == "HR" & composicion == "HR2" ~ 1,
  presidente == "HR" & composicion != "HR2" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_jn1 = case_when(presidente == "JN" & composicion == "JN1" ~ 1,
  presidente == "JN" & composicion != "JN1" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_jn2 = case_when(presidente == "JN" & composicion == "JN3" ~ 1,
  presidente == "JN" & composicion != "JN3" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_jn3 = case_when(presidente == "JN" & composicion == "JN5" ~ 1,
  presidente == "JN" & composicion != "JN5" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl1 = case_when(presidente == "RL" & composicion == "RL1" ~ 1,
  presidente == "RL" & composicion != "RL1" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl2 = case_when(presidente == "RL" & composicion == "RL2" ~ 1,
  presidente == "RL" & composicion != "RL2" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl3 = case_when(presidente == "RL" & composicion == "RL3" ~ 1,
  presidente == "RL" & composicion != "RL3" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl4 = case_when(presidente == "RL" & composicion == "RL4" ~ 1,
  presidente == "RL" & composicion != "RL4" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl5 = case_when(presidente == "RL" & composicion == "RL7" ~ 1,
  presidente == "RL" & composicion != "RL7" ~ 0, TRUE ~ NA))
df = df %>% mutate(composicion_rl6 = case_when(presidente == "RL" & composicion == "RL8" ~ 1,
  presidente == "RL" & composicion != "RL8" ~ 0, TRUE ~ NA))

# Features concerning the solicitator general
df$decision_procurador <- factor(tolower(df$decision_procurador))

# dummy variable indicating whether it issues an interim injunction for the solicitator general
df = df %>% mutate(involucra_recurso_interjur_procurador_1 = case_when(tolower(involucra_recurso_interjur_procurador) == "si" ~ 0,
                        tolower(involucra_recurso_interjur_procurador) == "no" ~ 1,
                        tolower(involucra_recurso_interjur_procurador) == "no dice" ~ 1,
                        TRUE ~ NA))
df = df %>% mutate(involucra_recurso_interjur_procurador_2 = case_when(tolower(involucra_recurso_interjur_procurador) == "si" ~ 0,
                        tolower(involucra_recurso_interjur_procurador) == "no" ~ 1,
                        tolower(involucra_recurso_interjur_procurador) == "no dice" ~ 0,
                        TRUE ~ NA))

# dummy variable indicating referral to the solicitator general
df$remision_al_procurador <- factor(tolower(df$remision_al_procurador))

### Models
### Court decision
df$decision_de_la_corte = as.factor(df$decision_de_la_corte)


## Models

In [ ]:
modelo1_vars <- c("año_de_presentacion","fecha_de_resolucion","tiempo_de_resolucion",
  "cita_de_fallos_dummy","cantidad_de_actores",
  "actor_empresa","actor_ente_nac",
  "actor_ente_prov","actor_mpfn","actor_municipio","actor_ong","actor_estado_nac",
  "actor_estado_prov","actor_pers_fisica","actor_pue_orig",
  "vecinos_de_otra_provincia_1","vecinos_de_otra_provincia_2",
  "cantidad_de_demandados",
  "demandado_caba","demandado_empresa","demandado_ente_nac",
  "demandado_ente_prov","demandado_estado_nac","demandado_estado_prov","demandado_municipio",
  "demandado_actor_interjuris","demandado_pers_fisica","demandado_empresa_estatal",
  "cantidad_de_provincias","citacion_de_terceros_1","citacion_de_terceros_2","provincias_citadas_terceros_tribunal_2",
  "tema","posterga_decision","decision_de_la_corte","involucra_recurso_interjur",
  "prueba_daño_interjur_2","dicta_cautelar","audiencia_publica","conformacion_mayoria",
  "disidencia_argibay","disidencia_highton","disidencia_lorenzetti",
  "voto_fayt","voto_petracchi","voto_highton","voto_argibay","voto_maqueda",
  "voto_zaffaroni","voto_rosenkrantz",
  'no_vota_argibay','no_vota_boggiano','no_vota_bossert',
  'no_vota_fayt','no_vota_highton','no_vota_levene','no_vota_lopez','no_vota_lorenzetti',
  'no_vota_maqueda','no_vota_moline','no_vota_nazareno','no_vota_petracchi','no_vota_rosatti',
  'no_vota_rosenkrantz','no_vota_vazquez','no_vota_zaffaroni',
  'composicion_cr1','composicion_emo1','composicion_ep1','composicion_ep2',
  'composicion_ep3','composicion_ep4','composicion_hr1','composicion_hr2',
  'composicion_jn1','composicion_jn2','composicion_jn3','composicion_rl1',
  'composicion_rl2','composicion_rl3','composicion_rl4','composicion_rl5','composicion_rl6',
  'involucra_recurso_interjur_procurador_1','involucra_recurso_interjur_procurador_2',"remision_al_procurador",
  "decision_procurador","presidente") #52

df_modelo1 <- df[modelo1_vars]

In [ ]:
set.seed(123)

# split into train and test
training_sample = sample(c(TRUE, FALSE), nrow(df_modelo1), replace = T, prob = c(0.80,0.20))
train = df_modelo1[training_sample, ]
test = df_modelo1[!training_sample, ]

### Models considering the complete dataset


---



In [ ]:
set.seed(32)
# Gini
modelo1_all_maximal_gini_32 <- rpart(decision_de_la_corte ~ ., data = df_modelo1,
  method = "class", control = rpart.control(minsplit = 2, cp=0), parms = list(split = 'gini'), usesurrogate = 2)



In [ ]:

bestcp_all_gini_32 <- modelo1_all_maximal_gini_32$cptable[which.min(modelo1_all_maximal_gini_32$cptable[,"xerror"]),"CP"]

modelo1_all_pruned_gini_32 <- prune(modelo1_all_maximal_gini_32, cp = bestcp_all_gini_32)

In [ ]:
pred.prune_gini_32 = predict(modelo1_all_pruned_gini_32, df_modelo1, type = "class")

#Confusion Matrix
conf.matrix <- table(df_modelo1$decision_de_la_corte, pred.prune_gini_32)
rownames(conf.matrix) <- paste("Actual", rownames(conf.matrix), sep = ":")
colnames(conf.matrix) <- paste("Pred", colnames(conf.matrix), sep = ":")
print(conf.matrix)

                pred.prune_gini_32
                 Pred:Acepta Pred:Rechaza
  Actual:Acepta           45            0
  Actual:Rechaza           2           67


In [ ]:
df_predict = df %>% filter(row_number() %in% as.numeric(rownames(df)))

expedientes_erroneos_df <- cbind(df_predict[c("expediente","decision_de_la_corte")], pred.prune_gini_32)
expedientes_erroneos_df %>% filter(decision_de_la_corte != pred.prune_gini_32)

,expediente,decision_de_la_corte,pred.prune_gini_32
,<chr>,<fct>,<fct>
13,A. 1032. XL. ORI,Rechaza,Acepta
31,P. 834. XXXIX. ORI,Rechaza,Acepta
